In [1]:
import pandas as pd
import json
import os
import gzip

In [2]:
os.chdir('./Data/biomart')
print(os.getcwd())

C:\RAG_LUMC\Data\biomart


In [3]:
%%bash
bash reorder_mart.sh

In [4]:
os.chdir('../..')

In [5]:
print(os.getcwd())

C:\RAG_LUMC


In [6]:


input_file = './Data/biomart/to_be_converted/wikipathways-20240910-gmt-Homo_sapiens.gmt'
output_file = './Data/biomart/to_be_converted/converted_wikipathways-20240910-gmt-Homo_sapiens.gmt'
json_file = './Data/JSON/ncbi_id_to_symbol.json'

# Load the gene ID to symbol dictionary from JSON
with open(json_file, 'r') as f:
    gene_dict = json.load(f)

# Read the GMT input file
columns = ['header', 'url'] + [f'gene_{i}' for i in range(1000)]  
df = pd.read_csv(input_file, sep='\t', header=None, names=columns, engine='python', dtype=str, na_filter=False)

# Function to replace gene IDs with symbols
def replace_gene_ids(gene_id):
    return gene_dict.get(gene_id, gene_id)

# Apply replacement function to gene columns
gene_columns = df.columns[2:]
for col in gene_columns:
    df[col] = df[col].apply(replace_gene_ids)

# Remove empty columns (if all values in a column are empty strings)
df = df.loc[:, (df != '').any(axis=0)]

# Remove empty rows for gene columns to prevent excess line breaks
df = df.apply(lambda x: x.dropna().tolist(), axis=1).apply(pd.Series)

# Save to the output file without excess newlines
df.to_csv(output_file, sep='\t', header=False, index=False, lineterminator='\n')

print(f'File conversion completed! Output saved to {output_file}')


File conversion completed! Output saved to ./Data/biomart/to_be_converted/converted_wikipathways-20240910-gmt-Homo_sapiens.gmt


In [7]:
input_file = './Data/biomart/to_be_converted/reordered_mart_export_2.txt.gz'  
output_json = './Data/JSON/genes.json' 
output_file = './Data/biomart/genes_consolidated.txt.gz'

In [8]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

ensure_dir(output_json)
ensure_dir(output_file)

df = pd.read_csv(input_file, compression='gzip')
print(f"Successfully read input file: {input_file}")

df['Gene Synonym'] = df['Gene Synonym'].fillna('').astype(str).str.strip()

df['Gene stable ID'] = df['Gene stable ID'].str.strip().str.upper()

unique_gene_ids = df['Gene stable ID'].nunique()
print(f"Number of unique 'Gene stable ID's: {unique_gene_ids}")

grouped = df.groupby('Gene stable ID').agg({
    'Gene name': 'first',
    'Gene description': 'first',
    'Gene Synonym': lambda x: sorted(filter(None, x.unique())),
    'NCBI gene (formerly Entrezgene) description': 'first',
    'NCBI gene (formerly Entrezgene) ID': 'first'
}).reset_index()

grouped.rename(columns={'Gene Synonym': 'Gene Synonyms'}, inplace=True)

grouped['Gene Synonyms'] = grouped['Gene Synonyms'].apply(lambda x: f"[{'_'.join(x)}]" if x else "[]")

consolidated_entries = grouped.shape[0]
print(f"Number of consolidated entries: {consolidated_entries}")

if consolidated_entries != unique_gene_ids:
    print("Warning: The number of consolidated entries does not match the number of unique 'Gene stable ID's.")
    print(f"Unique 'Gene stable ID's: {unique_gene_ids}, Consolidated entries: {consolidated_entries}")
else:
    print("Success: The number of consolidated entries matches the number of unique 'Gene stable ID's.")

genes_list = grouped.to_dict(orient='records')

with open(output_json, 'w', encoding='utf-8') as f_json:
    json.dump(genes_list, f_json, indent=4)
print(f"Consolidated JSON data has been saved to '{output_json}'.")

ordered_columns = [
    'Gene stable ID',
    'Gene name',
    'Gene description',
    'Gene Synonyms',
    'NCBI gene (formerly Entrezgene) description',
    'NCBI gene (formerly Entrezgene) ID'
]

missing_columns = set(ordered_columns) - set(grouped.columns)
if missing_columns:
    print(f"Error: Missing columns in the DataFrame: {missing_columns}")
    exit(1)

grouped_ordered = grouped[ordered_columns]

grouped_ordered.to_csv(output_file, index=False, sep=',', compression='gzip')
print(f"Consolidated TXT.GZ data has been saved to '{output_file}'.")


Successfully read input file: ./Data/biomart/to_be_converted/reordered_mart_export_2.txt.gz
Number of unique 'Gene stable ID's: 70611
Number of consolidated entries: 70611
Success: The number of consolidated entries matches the number of unique 'Gene stable ID's.
Consolidated JSON data has been saved to './Data/JSON/genes.json'.
Consolidated TXT.GZ data has been saved to './Data/biomart/genes_consolidated.txt.gz'.


In [9]:
genes_json_path = './Data/JSON/genes.json'  
input_gmt_path = './Data/biomart/to_be_converted/converted_wikipathways-20240910-gmt-Homo_sapiens.gmt'  
output_gmt_path = './Data/biomart/wikipathways_synonyms_Homo_sapiens.gmt.gz' 

In [10]:
ensure_dir(output_gmt_path)

# Load genes data
with open(genes_json_path, 'r', encoding='utf-8') as f_json:
    genes_data = json.load(f_json)

# Create gene to synonyms mapping
gene_to_synonyms = {}
for entry in genes_data:
    gene_name = entry.get('Gene name')
    if gene_name is None:
        continue
    gene_name = gene_name.strip()
    
    synonyms_str = entry.get('Gene Synonyms') or ''
    synonyms_str = synonyms_str.strip()
    if synonyms_str.startswith('[') and synonyms_str.endswith(']'):
        synonyms_str = synonyms_str[1:-1]
    
    synonyms = synonyms_str.split('_') if synonyms_str else []
    synonyms = [syn.strip() for syn in synonyms if syn.strip()]
    
    gene_to_synonyms[gene_name] = synonyms

print(f"Loaded {len(gene_to_synonyms)} genes with synonyms.")

# Process GMT file and save as .gmt.gz
def process_gmt(input_path, output_path, gene_synonyms_map):
    with open(input_path, 'r', encoding='utf-8') as infile, gzip.open(output_path, 'wt', encoding='utf-8') as outfile:
        for line_number, line in enumerate(infile, 1):
            line = line.strip()
            if not line:
                continue  
            parts = line.split('\t')
            if len(parts) < 3:
                print(f"Warning: Line {line_number} in GMT file does not have enough columns. Skipping.")
                continue
            pathway_name, pathway_url, *genes = parts
            expanded_genes = []
            for gene in genes:
                gene = gene.strip()
                if not gene:
                    continue  
                synonyms = gene_synonyms_map.get(gene, [])
                if synonyms:
                    expanded_gene = f"[{gene}, " + ", ".join(synonyms) + "]"
                else:
                    expanded_gene = f"[{gene}]"
                expanded_genes.append(expanded_gene)
            seen = set()
            unique_genes = []
            for gene in expanded_genes:
                if gene not in seen:
                    seen.add(gene)
                    unique_genes.append(gene)
            new_line = '\t'.join([pathway_name, pathway_url] + unique_genes)
            outfile.write(new_line + '\n')
            if line_number % 1000 == 0:
                print(f"Processed {line_number} lines.")
    print(f"Finished processing GMT file. Output saved to '{output_path}'.")

process_gmt(input_gmt_path, output_gmt_path, gene_to_synonyms)

Loaded 41067 genes with synonyms.
Finished processing GMT file. Output saved to './Data/biomart/wikipathways_synonyms_Homo_sapiens.gmt.gz'.
